In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 59.5 MB/s eta 0:00:00


EXP 3 : Z vs N (data brute )

In [3]:
import time
import numpy as np
import os
import pywt  # Wavelet library
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define a function to load the data from the directories
def load_data_from_directory_with_wavelet(directory_path, label, wavelet='db4', level=5):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory, including .txt and .TXT files
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)
        try:
            file_data = np.loadtxt(file_path)

            # Apply wavelet decomposition
            coeffs = pywt.wavedec(file_data, wavelet, level=level)

            # Divide the coefficients: Low frequency (approximation) and High frequency (details)
            cA5, cD5, cD4, cD3, cD2, cD1 = coeffs  # Decomposed levels

            # Flatten coefficients into 1D arrays for each component
            low_freq = cA5.flatten()  # Approximation coefficients at level 5
            high_freq = np.hstack([cD5.flatten(), cD4.flatten(), cD3.flatten(), cD2.flatten(), cD1.flatten()])  # High frequency coefficients

            # Append low and high frequency features as separate channels
            data.append(np.hstack([low_freq, high_freq]))  # Concatenate low and high frequency features
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (N vs S only)
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'  # Epileptic (N)
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'  # Seizure (S)

# Step 1: Load data with wavelet decomposition (N vs S)
data_N, labels_N = load_data_from_directory_with_wavelet(path_N, label=1, wavelet='db4', level=5)
data_S, labels_S = load_data_from_directory_with_wavelet(path_S, label=0, wavelet='db4', level=5)

# Step 2: Concatenate data and labels from the two classes (N vs S)
if data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_N, labels_S), axis=0)

    # Ensure the data is of type float32
    data_all = np.array(data_all.tolist(), dtype=np.float32)  # Convert object array to float32

    # Step 3: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # Step 4: Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    # Compile the model
    model.compile( loss='binary_crossentropy', metrics=['accuracy'])

    # Step 5: Start the timer
    start_time = time.time()

    # Step 6: Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Step 7: Evaluate the model on the test set
    y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary class labels

    # Step 8: Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Step 9: Calculate Specificity and Sensitivity
    tn, fp, fn, tp = conf_matrix.ravel()

    sensitivity = tp / (tp + fn)  # Sensitivity (True Positive Rate)
    specificity = tn / (tn + fp)  # Specificity (True Negative Rate)

    # Step 10: Print evaluation metrics
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)

    # Print the time taken
    end_time = time.time()
    print("Time taken for training and evaluation: ", end_time - start_time, "seconds")

else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


        # Define the model path
model_path = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp3/No_Options.keras"

# Save the model
model.save(model_path)

print(f"Model saved successfully at {model_path} ✅")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.5850 - loss: 192.4164 - val_accuracy: 0.7833 - val_loss: 19.5745
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8660 - loss: 7.1533 - val_accuracy: 0.9000 - val_loss: 5.0495
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9920 - loss: 0.0519 - val_accuracy: 0.8833 - val_loss: 4.9857
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 2.9192e-07 - val_accuracy: 0.8833 - val_loss: 4.9852
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 9.7196e-07 - val_accuracy: 0.8833 - val_loss: 4.9847
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 5.7583e-07 - val_accuracy: 0.8833 - val_loss: 4.9840
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 5.3540e-07 - val_accuracy: 0.8833 - val_loss: 4.9832
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 4.9700e-07 - val_accuracy: 0.8833 - val_l

With Standerscaler

In [6]:
import time
import numpy as np
import os
import pywt  # Wavelet library
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import StandardScaler  # Import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define a function to load the data from the directories
def load_data_from_directory_with_wavelet(directory_path, label, wavelet='db4', level=5):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory, including .txt and .TXT files
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)
        try:
            file_data = np.loadtxt(file_path)

            # Apply wavelet decomposition
            coeffs = pywt.wavedec(file_data, wavelet, level=level)

            # Divide the coefficients: Low frequency (approximation) and High frequency (details)
            cA5, cD5, cD4, cD3, cD2, cD1 = coeffs  # Decomposed levels

            # Flatten coefficients into 1D arrays for each component
            low_freq = cA5.flatten()  # Approximation coefficients at level 5
            high_freq = np.hstack([cD5.flatten(), cD4.flatten(), cD3.flatten(), cD2.flatten(), cD1.flatten()])  # High frequency coefficients

            # Append low and high frequency features as separate channels
            data.append(np.hstack([low_freq, high_freq]))  # Concatenate low and high frequency features
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (N vs S only)
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'  # Epileptic (N)
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'  # Seizure (S)

# Step 1: Load data with wavelet decomposition (N vs S)
data_N, labels_N = load_data_from_directory_with_wavelet(path_N, label=1, wavelet='db4', level=5)
data_S, labels_S = load_data_from_directory_with_wavelet(path_S, label=0, wavelet='db4', level=5)

# Step 2: Concatenate data and labels from the two classes (N vs S)
if data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_N, labels_S), axis=0)

    # Ensure the data is of type float32
    data_all = np.array(data_all.tolist(), dtype=np.float32)  # Convert object array to float32

    # Step 3: Scale the data using StandardScaler
    scaler = StandardScaler()
    data_all = scaler.fit_transform(data_all)  # Normalize the data

    # Step 4: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # Step 5: Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    # Compile the model
    model.compile( loss='binary_crossentropy', metrics=['accuracy'])

    # Step 6: Start the timer
    start_time = time.time()

    # Step 7: Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Step 8: Evaluate the model on the test set
    y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary class labels

    # Step 9: Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Step 10: Calculate Specificity and Sensitivity
    tn, fp, fn, tp = conf_matrix.ravel()

    sensitivity = tp / (tp + fn)  # Sensitivity (True Positive Rate)
    specificity = tn / (tn + fp)  # Specificity (True Negative Rate)

    # Step 11: Print evaluation metrics
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)

    # Print the time taken
    end_time = time.time()
    print("Time taken for training and evaluation: ", end_time - start_time, "seconds")

else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")

       # Define the model path
model_path = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp3/With_Stander.keras"

# Save the model
model.save(model_path)

print(f"Model saved successfully at {model_path} ✅")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.5430 - loss: 0.7634 - val_accuracy: 0.9000 - val_loss: 0.2026
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9890 - loss: 0.0667 - val_accuracy: 0.9167 - val_loss: 0.1438
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.0178 - val_accuracy: 0.9167 - val_loss: 0.1781
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 0.0048 - val_accuracy: 0.9167 - val_loss: 0.2068
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 0.0026 - val_accuracy: 0.9167 - val_loss: 0.2123
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 0.9167 - val_loss: 0.2319
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.9167 - val_loss: 0.2466
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 6.6180e-04 - val_accuracy: 0.9167 - val_loss: 0.2626
Epoch 9

With optmizer

In [4]:
import time
import numpy as np
import os
import pywt  # Wavelet library
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define a function to load the data from the directories
def load_data_from_directory_with_wavelet(directory_path, label, wavelet='db4', level=5):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory, including .txt and .TXT files
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)
        try:
            file_data = np.loadtxt(file_path)

            # Apply wavelet decomposition
            coeffs = pywt.wavedec(file_data, wavelet, level=level)

            # Divide the coefficients: Low frequency (approximation) and High frequency (details)
            cA5, cD5, cD4, cD3, cD2, cD1 = coeffs  # Decomposed levels

            # Flatten coefficients into 1D arrays for each component
            low_freq = cA5.flatten()  # Approximation coefficients at level 5
            high_freq = np.hstack([cD5.flatten(), cD4.flatten(), cD3.flatten(), cD2.flatten(), cD1.flatten()])  # High frequency coefficients

            # Append low and high frequency features as separate channels
            data.append(np.hstack([low_freq, high_freq]))  # Concatenate low and high frequency features
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (N vs S only)
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'  # Epileptic (N)
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'  # Seizure (S)

# Step 1: Load data with wavelet decomposition (N vs S)
data_N, labels_N = load_data_from_directory_with_wavelet(path_N, label=1, wavelet='db4', level=5)
data_S, labels_S = load_data_from_directory_with_wavelet(path_S, label=0, wavelet='db4', level=5)

# Step 2: Concatenate data and labels from the two classes (N vs S)
if data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_N, labels_S), axis=0)

    # Ensure the data is of type float32
    data_all = np.array(data_all.tolist(), dtype=np.float32)  # Convert object array to float32

    # Step 3: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # Step 4: Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Step 5: Start the timer
    start_time = time.time()

    # Step 6: Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Step 7: Evaluate the model on the test set
    y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary class labels

    # Step 8: Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Step 9: Calculate Specificity and Sensitivity
    tn, fp, fn, tp = conf_matrix.ravel()

    sensitivity = tp / (tp + fn)  # Sensitivity (True Positive Rate)
    specificity = tn / (tn + fp)  # Specificity (True Negative Rate)

    # Step 10: Print evaluation metrics
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)

    # Print the time taken
    end_time = time.time()
    print("Time taken for training and evaluation: ", end_time - start_time, "seconds")

else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")



      # Define the model path
model_path = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp3/With_Optimiser.keras"

# Save the model
model.save(model_path)

print(f"Model saved successfully at {model_path} ✅")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.5232 - loss: 37.8785 - val_accuracy: 0.6833 - val_loss: 20.2811
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8955 - loss: 6.9252 - val_accuracy: 0.7333 - val_loss: 10.5907
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9714 - loss: 0.5625 - val_accuracy: 0.6333 - val_loss: 18.1550
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9655 - loss: 0.4865 - val_accuracy: 0.7667 - val_loss: 11.8322
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 3.1930e-07 - val_accuracy: 0.8167 - val_loss: 10.3698
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 2.6421e-11 - val_accuracy: 0.8000 - val_loss: 12.9152
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 5.9130e-12 - val_accuracy: 0.8167 - val_loss: 16.0269
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 1.0000 - loss: 4.5056e-12 - val_accuracy: 0.7833 - val_

Both Optimiser and standarisation

In [5]:
import time
import numpy as np
import os
import pywt  # Wavelet library
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import StandardScaler  # Import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define a function to load the data from the directories
def load_data_from_directory_with_wavelet(directory_path, label, wavelet='db4', level=5):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory, including .txt and .TXT files
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)
        try:
            file_data = np.loadtxt(file_path)

            # Apply wavelet decomposition
            coeffs = pywt.wavedec(file_data, wavelet, level=level)

            # Divide the coefficients: Low frequency (approximation) and High frequency (details)
            cA5, cD5, cD4, cD3, cD2, cD1 = coeffs  # Decomposed levels

            # Flatten coefficients into 1D arrays for each component
            low_freq = cA5.flatten()  # Approximation coefficients at level 5
            high_freq = np.hstack([cD5.flatten(), cD4.flatten(), cD3.flatten(), cD2.flatten(), cD1.flatten()])  # High frequency coefficients

            # Append low and high frequency features as separate channels
            data.append(np.hstack([low_freq, high_freq]))  # Concatenate low and high frequency features
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (N vs S only)
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'  # Epileptic (N)
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'  # Seizure (S)

# Step 1: Load data with wavelet decomposition (N vs S)
data_N, labels_N = load_data_from_directory_with_wavelet(path_N, label=1, wavelet='db4', level=5)
data_S, labels_S = load_data_from_directory_with_wavelet(path_S, label=0, wavelet='db4', level=5)

# Step 2: Concatenate data and labels from the two classes (N vs S)
if data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_N, labels_S), axis=0)

    # Ensure the data is of type float32
    data_all = np.array(data_all.tolist(), dtype=np.float32)  # Convert object array to float32

    # Step 3: Scale the data using StandardScaler
    scaler = StandardScaler()
    data_all = scaler.fit_transform(data_all)  # Normalize the data

    # Step 4: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # Step 5: Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Step 6: Start the timer
    start_time = time.time()

    # Step 7: Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Step 8: Evaluate the model on the test set
    y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary class labels

    # Step 9: Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Step 10: Calculate Specificity and Sensitivity
    tn, fp, fn, tp = conf_matrix.ravel()

    sensitivity = tp / (tp + fn)  # Sensitivity (True Positive Rate)
    specificity = tn / (tn + fp)  # Specificity (True Negative Rate)

    # Step 11: Print evaluation metrics
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)

    # Print the time taken
    end_time = time.time()
    print("Time taken for training and evaluation: ", end_time - start_time, "seconds")

else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


       # Define the model path
model_path = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp3/With_Options.keras"

# Save the model
model.save(model_path)

print(f"Model saved successfully at {model_path} ✅")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.6372 - loss: 0.5241 - val_accuracy: 0.9667 - val_loss: 0.2519
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.1274 - val_accuracy: 0.8167 - val_loss: 0.3855
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.0113 - val_accuracy: 0.8000 - val_loss: 0.7440
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 3.0530e-04 - val_accuracy: 0.8333 - val_loss: 0.8770
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 2.5870e-05 - val_accuracy: 0.8167 - val_loss: 0.9662
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 5.4589e-06 - val_accuracy: 0.8333 - val_loss: 1.0257
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 2.3513e-06 - val_accuracy: 0.8333 - val_loss: 1.0637
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 1.0000 - loss: 1.2757e-06 - val_accuracy: 0.8333 - val_loss

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Accuracy:  0.8333333333333334
Recall:  1.0
Precision:  0.7560975609756098
Sensitivity:  1.0
Specificity:  0.6551724137931034
Time taken for training and evaluation:  8.03501582145691 seconds
Model saved successfully at /content/drive/MyDrive/Models/Walvet_dec/ANN/Exp3/With_Options.keras ✅
